<a href="https://colab.research.google.com/github/Samin-Sadaf7/QuestionAnswerGenerator/blob/main/PDFreader_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall typer -y
!pip install --force-reinstall spacy weasel

Found existing installation: typer 0.12.3
Uninstalling typer-0.12.3:
  Successfully uninstalled typer-0.12.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.9/156.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.4/922.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.0/493.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.9/423.9 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 14.3 MB/s eta 0:00:

In [2]:
!pip install --force-reinstall --no-deps typer==0.9.0
!pip install fastapi-cli
!pip install langchain-community sentence-transformers pypdf2 pypdf chromadb
!pip install google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
  Using cached typer-0.12.3-py3-none-any.whl (47 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.0
    Uninstalling typer-0.9.0:
      Successfully uninstalled typer-0.9.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 32.1 MB/s eta

In [3]:
!pip install pandas

In [4]:
!pip install --quiet pytesseract

In [5]:
!pip install "unstructured[all-docs]" pillow pydantic lxml matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.4/466.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.8 MB/s et

In [6]:
!sudo apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,410 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archive.ub

In [7]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.4 [186 kB]
Fetched 186 kB in 0s (1,153 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 12357

In [8]:
!sudo apt-get install libleptonica-dev tesseract-ocr libtesseract-dev python3-pil tesseract-ocr-script-latn

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libimagequant0 libraqm0 mailcap mime-support python3-olefile
  tesseract-ocr-eng tesseract-ocr-osd
Suggested packages:
  python-pil-doc
The following NEW packages will be installed:
  libarchive-dev libimagequant0 libleptonica-dev libraqm0 libtesseract-dev
  mailcap mime-support python3-olefile python3-pil tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd tesseract-ocr-script-latn
0 upgraded, 13 newly installed, 0 to remove and 45 not upgraded.
Need to get 40.0 MB of archives.
After this operation, 123 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libimagequant0 amd64 2.17.0-1 [34.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-d

In [9]:
!pip install unstructured-pytesseract
!pip install tesseract-ocr

  Preparing metadata (setup.py) ... done
  Created wheel for tesseract-ocr: filename=tesseract_ocr-0.0.1-cp310-cp310-linux_x86_64.whl size=169756 sha256=28f10ed3a2c0603af29b8a1ab475c7442d530b7f6b949cc17ff278324be47e1d
  Stored in directory: /root/.cache/pip/wheels/bb/fd/f3/5c231ecbbb80a1fe33204ff3021d99b54ef6daf6f8099311b8
Successfully built tesseract-ocr


In [34]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
import os
import textwrap

In [35]:
def split_pdf(file_path, chunk_size, chunk_overlap):
    loader = PyPDFLoader(file_path)
    pages = loader.load_and_split()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(pages)
    return docs

In [36]:
!pip install --quiet pdf2image pytesseract pillow

In [37]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.4).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [1]:
from unstructured.partition.pdf import partition_pdf

In [2]:
from pdf2image import convert_from_path
from PIL import Image
def pdf_to_images(pdf_path, first_page = 1, last_page = 5):
    images = convert_from_path(pdf_path, first_page=first_page, last_page=last_page)
    for image in images[:1]:
        Image.fromarray(image).show()
    return images

In [3]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import tempfile
import os
from tqdm import tqdm

def pdf_to_images(pdf_path, first_page=1, last_page=100):
    images = convert_from_path(pdf_path, first_page=first_page, last_page=last_page)
    return images

def images_to_text(images):
    text_list = []
    for idx, image in enumerate(tqdm(images)):
        # Save each image as a temporary file
        temp_image_path = f"temp_image_{idx}.png"
        image.save(temp_image_path, format='PNG')

        # Use pytesseract to perform OCR on the temporary image file
        text = pytesseract.image_to_string(temp_image_path)
        text_list.append(text.strip())  # Strip any leading/trailing whitespace

        # Clean up temporary file
        os.remove(temp_image_path)

    return text_list

In [4]:
def extract_text_from_pdf(pdf_path):
    images = pdf_to_images(pdf_path)
    texts = images_to_text(images)
    return texts

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#PDF Extraction

In [6]:
pdf_path = '/content/gdrive/MyDrive/AI_integration_kernel/MediBee_MBBS_AI/All Prof Course/1st Prof/Anatomy/Guide/(rej)Arif Anatomy Volume I.pdf'

In [ ]:
res_pdf_elements =partition_pdf(
    filename =pdf_path,
    strategy="hi_res",
    extract_images_in_pdf=True,
    extract_image_block_types=["Image"],
    extract_image_block_to_payload=False,
    extract_image_block_output_dir="extracted_data_image"
)

In [ ]:
img =[]
for element in res_pdf_elements:
  if "unstructured.documents.elements.Image" in str(type(element)):
    img.append(str(element))

In [ ]:
from PIL import Image
for img_path in img[:1]:
  img = Image.open(img_path)
  img.show()

In [ ]:
docs =split_pdf(pdf_path, chunk_size=3000, chunk_overlap=500)

In [7]:
!pip install pdf2image pytesseract torch torchvision

In [8]:
!pip install numpy

In [9]:
import numpy as np

In [10]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
import torch
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
from tqdm import tqdm
import os
from multiprocessing import Pool, Manager
import numpy as np
from PyPDF2 import PdfReader

# Function to get the total number of pages in the PDF
def get_total_pages(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf = PdfReader(f)
        return len(pdf.pages)

# Function to convert a batch of PDF pages to images
def pdf_to_images_batch(pdf_path, first_page, last_page):
    images = convert_from_path(pdf_path, first_page=first_page, last_page=last_page)
    return images

# Function to process a batch of images and extract text
def process_batch(batch_info, result_list):
    pdf_path, page_numbers, device = batch_info

    text_list = []
    for page_number in tqdm(page_numbers, desc=f"Processing batch on device {device}"):
        # Convert the specific page to an image
        images = pdf_to_images_batch(pdf_path, first_page=page_number, last_page=page_number)
        image = images[0]  # Only one image since we're processing one page at a time

        # Save the image as a temporary file
        temp_image_path = f"temp_image_{device}_{page_number}.png"
        image.save(temp_image_path, format='PNG')

        # Use pytesseract to perform OCR on the temporary image file
        text = pytesseract.image_to_string(temp_image_path)
        text_list.append((page_number, text.strip()))  # Strip any leading/trailing whitespace

        # Clean up temporary file
        os.remove(temp_image_path)

    result_list.append(text_list)

def extract_text_from_pdf(pdf_path, num_batches=10):
    # Check if CUDA (GPU) is available
    use_cuda = torch.cuda.is_available()

    # Get the total number of pages in the PDF
    total_pages = get_total_pages(pdf_path)

    # Calculate the number of pages per batch
    pages_per_batch = (total_pages + num_batches - 1) // num_batches  # Ensure we cover all pages

    # Create batches
    batches = []
    for i in range(num_batches):
        start_page = i * pages_per_batch + 1
        end_page = min((i + 1) * pages_per_batch, total_pages)
        page_numbers = list(range(start_page, end_page + 1))

        # Choose device (GPU or CPU)
        if use_cuda:
            device = torch.device('cuda', i % torch.cuda.device_count())
        else:
            device = torch.device('cpu')

        batches.append((pdf_path, page_numbers, device))

    # Use multiprocessing to process batches in parallel
    manager = Manager()
    result_list = manager.list()
    with Pool(processes=len(batches)) as pool:
        for batch in batches:
            pool.apply_async(process_batch, args=(batch, result_list))

        # Close the pool and wait for all processes to finish
        pool.close()
        pool.join()

    # Combine results from result_list
    extracted_texts = [text for batch_result in result_list for text in batch_result]
    extracted_texts.sort(key=lambda x: x[0])

    # Return only the texts, discarding the page numbers
    return [text for _, text in extracted_texts]

extracted_texts = extract_text_from_pdf(pdf_path, num_batches=10)

for page_number, text in enumerate(extracted_texts, start=1):
    print(f"Page {page_number}:\n{text}\n")

Processing batch on device cpu: 100%|██████████| 62/62 [1:36:20<00:00, 93.24s/it]


Streaming output truncated to the last 5000 lines.
 

Fe Orga and incrton o bess bach and cracobrachiaie. >

Page 558:
Origin from front of shaft
of humerus and from
medial and lateral
intermuscular septa

Insertion into front of
coronoid process and
tuberosity of ulna

 

Fig: The origin and insertion of the brachialis muscle. Fig: The triceps brachii muscle.

Nice to Know:
Anatomical events at the insertion of coracobrachialis: It is an important landmark as many anatomical events
occur at this level (Fig. below):

Median nerve

Anterior
compartment

    

Brachial artery
Ulnar nerve
Deltoid — Coracobrachialis
Medial
Lateral
ntermuscular
iritermuscular
septum

septum

Radial nerve

Posterior compartment

Fig: Anatomical events at the insertion of coracobrachialis. a

Circular shaft of the humerus becomes triangular below this level,

Brachial artery passes from medial side of arm to its anterior aspect,

Basilic vein pierces the deep fascia.

Median nerve crosses in front of the brac

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_pdf_file.pdf'

In [11]:
!pip install --quiet reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.6 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def text_list_to_pdf(text_list, pdf_path):
    # Create a canvas object
    c = canvas.Canvas(pdf_path, pagesize=letter)
    width, height = letter

    # Set up the font and size
    c.setFont("Helvetica", 12)

    # Define the position to start the text
    x, y = 50, height - 50

    # Write each line from the list to the PDF
    for line in text_list:
        c.drawString(x, y, line)
        y -= 15  # Move to the next line
        # Check if the y position is at the bottom of the page
        if y < 50:
            c.showPage()  # Create a new page
            c.setFont("Helvetica", 12)
            y = height - 50

    # Save the PDF file
    c.save()

In [ ]:
def save_text_list_to_file(text_list, file_path):
    with open(file_path, 'w') as file:
        for line in text_list:
            file.write(line + '\n')

In [ ]:
file_path = "output.txt"
save_text_list_to_file(extracted_texts, file_path)

In [ ]:
pdf_path = "output.pdf"
text_list_to_pdf(extracted_texts, pdf_path)

#Data to MongoDB

In [14]:
import pandas as pd

In [ ]:
# Prepare data for DataFrame
data = []
for doc in docs:
    data.append({
        'text': doc.page_content,
        'source': doc.metadata['source'],
        'page': doc.metadata['page'],
    })

# Create DataFrame
df = pd.DataFrame(data)

In [15]:
!pip install --quiet pandas pymongo sentence_transformers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00


In [16]:
!python -m pip install "pymongo[srv]"

In [17]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

from google.colab import userdata
uri = userdata.get('MongoDB_URL')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [18]:
db = client['RAG_application']

#Update Medical Books RAG database

In [ ]:
db.Medical_Books_RAG.distinct('source')

['/content/drive/MyDrive/All Prof Course/1st Prof/Anatomy/Text Book/Vishram Singh Textbook of Anatomy  Head, Neck, and Brain..pdf',
 '/content/drive/MyDrive/All Prof Course/1st Prof/Anatomy/Text Book/vishram singh textbook of anatomy abdomen and lower limb.pdf',
 '/content/drive/MyDrive/All Prof Course/2nd Prof/Microbiology/Text Book/Lange Medical Microbiology 28th Edition.pdf',
 '/content/drive/MyDrive/All Prof Course/2nd Prof/Pharmacology/Text Book/Basic & Clinical Pharmacology (Katzung).pdf',
 '/content/gdrive/MyDrive/AI_integration_kernel/Reddy Forensic 33rd.pdf']

In [ ]:
updated_results = db.Medical_Books_RAG.update_many({'source': '/content/gdrive/MyDrive/AI_integration_kernel/Reddy Forensic 33rd.pdf'}, {'$set': {'subject': 'Forensic Medicine'}})
print(f"Updated {updated_results.modified_count} documents")

Updated 1821 documents


#Add CSV File into it

In [ ]:
emb_csv = pd.read_csv("/content/gdrive/MyDrive/AI_integration_kernel/MediBee_MBBS_AI/All Prof Course/1st Prof/Anatomy/Text Book/vishram singh textbook of anatomy abdomen and lower limb.csv")

In [ ]:
emb_df =pd.DataFrame(emb_csv)
emb_df.head()

,Unnamed: 0,text,source,page,embedding
0,0,"T extbook of Anatomy: Abdomen and Lower Limb, ...",/content/drive/MyDrive/All Prof Course/1st Pro...,4,"[0.0012812747154384851, 0.0028495092410594225,..."
1,1,Preface to the \nSecond Edition\nIt is with gr...,/content/drive/MyDrive/All Prof Course/1st Pro...,7,"[-0.014985816553235054, 0.0016522323712706566,..."
2,2,Preface to the \nFirst Edition\nThis textbook ...,/content/drive/MyDrive/All Prof Course/1st Pro...,9,"[-0.013375895097851753, 0.00621607294306159, 0..."
3,3,"Acknowledgments\nAt the outset, I express my g...",/content/drive/MyDrive/All Prof Course/1st Pro...,11,"[0.008156468160450459, 0.0010366947390139103, ..."
4,4,1Introduction and \nOverview of the \nAbdomenC...,/content/drive/MyDrive/All Prof Course/1st Pro...,15,"[0.0002767481200862676, -0.0006620225613005459..."


In [ ]:
emb_df['embedding'].dtype

dtype('O')

In [ ]:
import ast

def str_to_list(s):
    return ast.literal_eval(s)

emb_df['embedding'] =emb_df['embedding'].apply(str_to_list)

emb_df.head()

,Unnamed: 0,text,source,page,embedding
0,0,"T extbook of Anatomy: Abdomen and Lower Limb, ...",/content/drive/MyDrive/All Prof Course/1st Pro...,4,"[0.0012812747154384851, 0.0028495092410594225,..."
1,1,Preface to the \nSecond Edition\nIt is with gr...,/content/drive/MyDrive/All Prof Course/1st Pro...,7,"[-0.014985816553235054, 0.0016522323712706566,..."
2,2,Preface to the \nFirst Edition\nThis textbook ...,/content/drive/MyDrive/All Prof Course/1st Pro...,9,"[-0.013375895097851753, 0.00621607294306159, 0..."
3,3,"Acknowledgments\nAt the outset, I express my g...",/content/drive/MyDrive/All Prof Course/1st Pro...,11,"[0.008156468160450459, 0.0010366947390139103, ..."
4,4,1Introduction and \nOverview of the \nAbdomenC...,/content/drive/MyDrive/All Prof Course/1st Pro...,15,"[0.0002767481200862676, -0.0006620225613005459..."


In [ ]:
for c in emb_df.columns:
  print(emb_df[c].dtype)

int64
object
object
int64
object


In [ ]:
emb_df.rename(columns={'page_content': 'text'}, inplace=True)

In [ ]:
emb_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
emb_df['flag'] =0

In [ ]:
emb_df.count()

text         501
source       501
page         501
embedding    501
flag         501
dtype: int64

In [ ]:
book_documents = emb_df.to_dict('records')
result = db.Medical_Books_RAG.insert_many(book_documents)
print(f"Inserted {len(result.inserted_ids)} documents into the collection.")

Inserted 501 documents into the collection.


In [ ]:
{
    "fields":[{
        "numDimensions" : 1024,
        "path" : "embedding",
        "similarity": "cosine",
        "type": "vector"
    }]
}

{'fields': [{'numDimensions': 1024,
   'path': 'embedding',
   'similarity': 'cosine',
   'type': 'vector'}]}

#Embedding

In [19]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("thenlper/gte-large")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [20]:
def get_embedding(text:str)->list[float]:
  if not text.strip():
    print("Attemted to do embedding on an empty text")
    return []
  embedding = embedding_model.encode(text)
  return embedding.tolist()

In [21]:
def vector_search(user_query, collection):
    query_embedding = get_embedding(user_query)
    print(query_embedding)

    if query_embedding is None:
        return "Invalid Query Embedding"

    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 10,
                "limit": 2,
            }
        },
        {
            "$project": {
                "text": 1,
                "flag": 1,
                "_id": 1,
                "score": {"$meta": "vectorSearchScore"},
            }
        },
        {
            "$match": {
                "flag": 0,
                "$expr": {
                    "$gte": [{"$strLenCP": "$text"}, 100]
                }
            }
        },
    ]

    # Run the aggregation pipeline
    result = list(collection.aggregate(pipeline))

    # Extract document IDs from the result
    document_ids = [doc["_id"] for doc in result]

    # Update the flag value to 1 for the matched documents
    if document_ids:
        update_result = collection.update_many(
            {"_id": {"$in": document_ids}},
            {"$set": {"flag": 1}}
        )
        print(f"Updated {update_result.modified_count} documents")

    return result

In [22]:
def get_knowledge(user_query, collection):
  get_knowledge = vector_search(user_query, collection)
  return get_knowledge

In [23]:
def get_search_result(get_knowledge):
  search_result = ""

  for result in get_knowledge:
    score = result.get('score')
    print(score)
    text = result.get('text',' ')
    text = textwrap.fill(text, width=100)
    print(text)
    text_length = len(text)
    print(text_length)
    if score > 0.8 and text_length > 200:
     search_result += f"{text}\n"

  return search_result

In [ ]:
df['embedding'] = df['text'].apply(get_embedding)

In [ ]:
df.columns

RangeIndex(start=0, stop=0, step=1)

In [ ]:
documents = df.to_dict('records')

In [ ]:
df.to_csv('/content/gdrive/MyDrive/AI_integration_kernel/MediBee_MBBS_AI/All Prof Course/2nd Prof/Forensic Medicine/Text Book/reddy_forensic_documents.csv', index=False)

In [ ]:
len(documents)

1821

# Find Data from Mongo

In [26]:
book_collection = db['Medical_Books_RAG']

In [27]:
results = list(db.topic.find({"subject":"Forensic Medicine"}))
topic_list = []
for result in results:
  topic_list.append(result)

In [28]:
len(topic_list)

37

In [29]:
question_topics = db.viva_question_collection.distinct("topic")
len(question_topics)

29

In [ ]:
query = ' Head neck: pharyngeal arches'
knowledge = get_knowledge(query, book_collection)
context = get_search_result(knowledge)
context

[-0.0022918186150491238, 0.008685228414833546, -0.017084909602999687, 0.009746587835252285, 0.01889963075518608, -0.018930120393633842, -0.006156123708933592, 0.03561919927597046, 0.027654284611344337, 0.05163608491420746, 0.04634861648082733, -0.012617578729987144, 0.012248323298990726, -0.038723696023225784, -0.027578089386224747, 0.020923208445310593, -0.039142705500125885, -0.03837312012910843, -0.01940949633717537, 0.024770962074398994, -0.00037492476985789835, -0.01055862382054329, -0.07302655279636383, 0.007441373076289892, -0.0001338961155852303, 0.03094548173248768, 0.015771090984344482, -0.0007005534134805202, 0.040412019938230515, 0.056433528661727905, -0.0057858373038470745, -0.0016936162719503045, 0.023819440975785255, -0.05216193199157715, -0.02239096723496914, -0.049870897084474564, 0.030964676290750504, -0.06639441847801208, 0.007878364063799381, -0.04956795647740364, 0.021987777203321457, -0.017063412815332413, 0.007371310610324144, -0.006488760933279991, -0.0172690991

'Pharynx and Palate 217 Golden Facts to Remember  /g34Killian’s dehiscence A weak area in the\nposterior wall of pharynx  between the two parts of inferior constrictor muscle  /g34Principal\nartery of tonsil Tonsillar branch of the facial artery  /g34Common source of bleeding after\ntonsillectomy Paratonsillar vein  /g34Tonsillar lymph node Jugulodigastric lymph node  /g34Most\ncommon variety of cleft palate Unilateral cleft palate involving the hard palate  /g34Largest\ntonsils Palatine tonsils  /g34Quinsy Chronic infection of peritonsillar space  /g34Coffin corner\nLingual tonsillar sulcus Case 1 A 47-year-old man came in the emergency OPD and  complained that\nwhile eating fish something got stuck  in his throat. It was causing pain and lot of discomfort.\nThe physical examination of throat revealed that  discomfort increases on moving the thyroid\ncartilage  from side to side. The physician concluded that the fish  bone was stuck in the piriform\nfossa. Questions 1. What is pirifor

In [ ]:
document_ids = [doc["_id"] for doc in knowledge]

# Update the flag value to 1 for the matched documents
if document_ids:
  update_result = book_collection.update_many(
            {"_id": {"$in": document_ids}},
            {"$set": {"flag": 0}}
   )
  print(f"Updated {update_result.modified_count} documents")

Updated 2 documents


In [ ]:
from pymongo import MongoClient
from pymongo.errors import CollectionInvalid

# Define the collection name
topic_name = 'topic'

# Define the question schema
topic_schema = {
    'prof': {
        'type': 'string',
        'required': True,
    },
    'term':{
        'type': 'string',
    },
    'subject': {
        'type': 'string',
    },
    'card': {
        'type': 'string',
        'required': True,
    },
    'topic': {
        'type': 'string',
        'required': True,
    },
    'sub_topic': {
        'type': 'string',
        'required': True,
    }
}

# Initialize the validator
validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'properties': {}
    }
}

required = []

def process_field(field_key, field, parent_required, parent_properties):
    properties = {'bsonType': field['type']}
    if 'min_length' in field:
        properties['minLength'] = field['min_length']
    if 'required' in field and field['required']:
        parent_required.append(field_key)
    parent_properties[field_key] = properties

for field_key, field in topic_schema.items():
    process_field(field_key, field, required, validator['$jsonSchema']['properties'])

if required:
    validator['$jsonSchema']['required'] = required


# Create the collection with the validator
try:
    db.create_collection(topic_name, validator=validator)
    print(f"Collection '{topic_name}' created with validator.")
except CollectionInvalid:
    print(f"Collection '{topic_name}' already exists or validation failed.")

print(validator)

Collection 'topic' already exists or validation failed.
{'$jsonSchema': {'bsonType': 'object', 'properties': {'prof': {'bsonType': 'string'}, 'term': {'bsonType': 'string'}, 'subject': {'bsonType': 'string'}, 'card': {'bsonType': 'string'}, 'topic': {'bsonType': 'string'}, 'sub_topic': {'bsonType': 'string'}}, 'required': ['prof', 'card', 'topic', 'sub_topic']}}


In [ ]:
collection = db['topic']

In [ ]:
import re
topic = 'Thoracic wall, intercostal space, thoracic inlet'
regex = re.compile(rf'^\s*{topic}\s*$', re.IGNORECASE)

# Query to find documents where the topic or subtopic matches the regex
query = {
    "$or": [
        {"topic": regex},
        {"subtopic": regex}
    ]
}

# Execute the query and save the results into a list
results = list(collection.find(query))
print(results)

[{'_id': ObjectId('668cce2d449c846bbcf10b60'), 'prof': 'First', 'subject': 'Anatomy', 'card': 'Thorax ', 'topic': ' Thoracic wall, intercostal space, thoracic inlet', 'subTopic': ' Thoracic wall, intercostal space, thoracic inlet'}]


In [ ]:
from pymongo.errors import CollectionInvalid

# Define the collection name
question_collection_name = 'viva_question_collection'

# Define the question schema
question_schema = {
    'prof': {
        'type': 'string',
        'required': True,
    },
    'subject': {
        'type': 'string',
        'required': True,
    },
    'card': {
        'type': 'string',
        'required': True,
    },
    'version': {
        'type': 'string',
        'required': True,
    },
    'topic': {
        'type': 'string',
        'required': True,
    },
    'question': {
        'type': 'string',
        'required': True,
    },
    'hint': {
        'type': 'string',
        'required': True,
    },
    'subTopic':{
        'type': 'string',
    },
    'correct_answer': {
        'type': 'string',
        'required': True,
    },
    'extra': {
        'oneof': [
            {
                'type': 'string',
            },
            {
                'type': 'null',
            },
        ],
        'required': True,
    }
}

# Initialize the validator
validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'properties': {}
    }
}

required = []

def process_field(field_key, field, parent_required, parent_properties):
    if 'oneof' in field:
        properties = {'bsonType': []}
        for option in field['oneof']:
            properties['bsonType'].append(option['type'])
    else:
        properties = {'bsonType': field['type']}

    if 'min_length' in field:
        properties['minLength'] = field['min_length']

    if 'required' in field and field['required']:
        parent_required.append(field_key)

    parent_properties[field_key] = properties

for field_key, field in question_schema.items():
    process_field(field_key, field, required, validator['$jsonSchema']['properties'])

if required:
    validator['$jsonSchema']['required'] = required

# Create the collection with the validator
try:
    db.create_collection(question_collection_name, validator=validator)
    print(f"Collection '{question_collection_name}' created with validator.")
except CollectionInvalid:
    print(f"Collection '{question_collection_name}' already exists or validation failed.")

print(validator)

Collection 'viva_question_collection' already exists or validation failed.
{'$jsonSchema': {'bsonType': 'object', 'properties': {'prof': {'bsonType': 'string'}, 'subject': {'bsonType': 'string'}, 'card': {'bsonType': 'string'}, 'version': {'bsonType': 'string'}, 'topic': {'bsonType': 'string'}, 'question': {'bsonType': 'string'}, 'hint': {'bsonType': 'string'}, 'subTopic': {'bsonType': 'string'}, 'correct_answer': {'bsonType': 'string'}, 'extra': {'bsonType': ['string', 'null']}}, 'required': ['prof', 'subject', 'card', 'version', 'topic', 'question', 'hint', 'correct_answer', 'extra']}}


#Prompts

In [30]:
def get_prompt_viva(query, context):
    context = context.replace("'", "").replace('"', "").replace('\n', "")
    prompt = (
        f"""You are assisting a medical student in preparing for an exam. Your task is to generate a viva question based on the given MCQ, Multiple Choice Question, Single Best answer, SQA question and True-False Question using \
medical terminology. Ensure your responses are comprehensive and friendly, reflecting a conversational tone. Your only focus is to generate Viva Question and you are given the Question Format. \
Make sure to generate questions in full sentence. Your task is to create the viva question.  \
Question should be answerable in 1-2 sentences. You have 5 tasks here: \
1. You can generate as many questions you can. \
2. You should provide corresponding hints for those questions. \
3. You should also provide answers for those questions. Answers should be around 2-3 sentences. \
4. You must provide extra information for the questions. If you do not find anything to add give null \
5. Output should be a perfect json object with proper format. \
Do not add any extra characters. \
The output should be a JSON object in the following format: (Output only the json object nothing else, not even a single extra character)
{{
    "Questions": [
        {{
            "Question": "Q1?",
            "Hint": "H1",
            "Answer": "A1",
            "Extra": "E1"
        }}
    ]
}}
The response should strictly be a  JSON object. Just a JSON object in the above format. Maintain the mentioned format strictly. Replace\
Based on the following question format.
Question Format:
Q1 will be replaced by actual question created from the context. H1 will be replaced by the relevant hint to answer the question. A1 will be replaced by the actual answer to the question. \
E1 will be replaced by extra information. You need to generate actual questions in this format using the following context. Number of questions will be 10. \
TOPIC: '{query}'
CONTEXT: '{context}'"""
    )
    return prompt

In [31]:
def get_answer_prompt(actual_answer, question, answer):
  actual_answer = actual_answer.replace("'", "").replace('"', "").replace('\n', "")
  prompt = ("""`You are assisting a medical student in preparing for an exam. The student has answered a QUESTION and want to check if the ANSWER is correct\
  As an assistant your job is to examine the ANSWER and provide a feedback. You are given the QUESTION and the student's ANSWER to that QUESTION. Also, the Actual Answer of that question is given.\
  Now evaluate the student's answer and compare it with the correct answer given below. Give a detailed feedback in first person under 100 words. BE STRICT about the evaluation and criticise the given answer properly. Generate normal text. No need to make the text bold.If the answer is incorrect\
  also provide the correct answer.\
  QUESTION: '{question}'
  Correct Answer: '{actual_answer}',
  STUDENT'S ANSWER: '{answer}'
  YOUR FEEDBACK:  """).format(actual_answer=actual_answer, question=question, answer=answer)
  return prompt

#Gemini Run

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GeminiKey')

In [ ]:
import google.generativeai as genai
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(model_name='gemini-pro')

In [ ]:
len(topic_list)

37

In [ ]:
for topic in topic_list[:4]:
  print(topic)

{'_id': ObjectId('668cce2d449c846bbcf10d1d'), 'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Legal procedures', 'subTopic': 'Legal procedures'}
{'_id': ObjectId('668cce2d449c846bbcf10d2a'), 'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Thermal injury', 'subTopic': 'Thermal injury'}
{'_id': ObjectId('668cce2d449c846bbcf10d1c'), 'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Introduction &amp; history of forensic medicine', 'subTopic': 'Introduction &amp; history of forensic medicine'}
{'_id': ObjectId('668cce2d449c846bbcf10d2c'), 'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Identification part-1', 'subTopic': 'Identification part-1'}


In [ ]:
import time
import json

questions_list = []

for topic in topic_list[:4]:
    query = topic['topic']

    try:
        flag = False
        #for topic in question_topics:
         #   if topic == query:
          #      flag = True
        # Get the context and prompt
        if flag == False:
          knowledge = get_knowledge(query, book_collection)
          context = get_search_result(knowledge)
          print("Context: ", context)
          prompt = get_prompt_viva(query=query, context=context)

          # Print input tokens
          print('Input tokens: ', model.count_tokens(prompt))

          # Generate the question and print it
          question = model.generate_content(prompt).text
          print(question, '\n\n')
          # Print output tokens
          print('Output tokens: ', model.count_tokens(question))
          # Add the question to the list
          questions_list.append(question)
    except ValueError as e:
        print(f"Error generating content for topic '{topic}': {e}")
        continue

    # Add a 10-second wait time
    time.sleep(20)

[-0.005878961645066738, -0.0019749202765524387, -0.013784551993012428, 0.005856963340193033, 0.003015558933839202, 0.001519199344329536, -0.0393468476831913, 0.00613044761121273, -0.0050398907624185085, 0.06839580088853836, 0.038677528500556946, 0.028130538761615753, 0.016563396900892258, -0.02720520831644535, -0.01797180250287056, 0.030569011345505714, -0.03009190782904625, -0.041836127638816833, 0.012217352166771889, 0.015966324135661125, -0.0012074249098077416, 0.01940840668976307, -0.06304574757814407, -0.015600555576384068, -0.005698140244930983, 0.019260220229625702, 0.03087109886109829, -0.03273424506187439, 0.07951512187719345, 0.035525109618902206, -0.042017124593257904, -0.025990406051278114, 0.02119935303926468, -0.017890634015202522, -0.02196314185857773, -0.033029403537511826, 0.05934511125087738, -0.04347669705748558, 0.00958603248000145, -0.030851304531097412, 0.009029517881572247, 0.016881445422768593, 0.03852375969290733, -0.04825221374630928, -0.06388333439826965, -0.

In [ ]:
document_ids = [doc["_id"] for doc in knowledge]

# Update the flag value to 1 for the matched documents
if document_ids:
  update_result = book_collection.update_many(
            {"_id": {"$in": document_ids}},
            {"$set": {"flag": 0}}
   )
  print(f"Updated {update_result.modified_count} documents")

Updated 0 documents


In [62]:
query = {"flag": 1}

    # Update the flag to 0
new_values = {"$set": {"flag": 0}}

    # Perform the update
results = db.Medical_Books_RAG.update_many(query, new_values)
print(f"Updated {results.modified_count} documents")

Updated 8 documents


In [ ]:
def json_correction_prompt(question_json):
    prompt = ("""Clean up the following json and make it in a proper format. You are given a JSON object that may contain errors in formatting, such as missing or misplaced commas, incorrect usage of brackets, or improper key-value pairs. Your task is to correct the JSON so that it is valid and properly formatted.
    Output Format is only a json object. Nothing else. \
    Do not add any extra characters.
    {question_json}
    """).format(question_json=question_json)
    return prompt

In [ ]:
import json

# Function to convert JSON to a list of documents
def convert_json_to_documents(topic_name, questions):
    documents = []
    for question in question_json:
        if len(questions) > 0:
            for question in questions:
                doc = {
                    'prof': topic_name['prof'],
                    'subject': topic_name['subject'],
                    'card': topic_name['card'],
                    'topic': topic_name['topic'],
                    'subTopic': topic_name['subTopic'],
                    'question': question['Question'],
                    'version': '2',
                    'hint': question['Hint'],
                    'correct_answer': question['Answer'],
                    'extra': question.get('Extra', "No extra info to add")
                }
                documents.append(doc)
    return documents

In [ ]:
import re
import json

def extract_json(text):
    # Regex to match JSON objects
    json_regex = re.compile(r'\{(?:[^{}]*|\{(?:[^{}]*|\{[^{}]*\})*\})*\}')

    # Function to recursively parse JSON objects
    def parse_json(match):
        try:
            json_object = json.loads(match)
            # Check if there are nested JSON objects
            for key, value in json_object.items():
                if isinstance(value, str) and re.match(json_regex, value):
                    json_object[key] = parse_json(value)
            return json_object
        except json.JSONDecodeError:
            return None

    # Find all JSON objects in the text
    json_matches = json_regex.findall(text)

    # Parse the JSON objects
    json_objects = []
    for json_match in json_matches:
        json_object = parse_json(json_match)
        if json_object is not None:
            json_objects.append(json_object)

    return json_objects

In [ ]:
docs = []
question_jsons = []
for question in questions_list:
        question_text = f"[{question}]"
        new_question_jsons = extract_json(question_text)
        question_jsons.append(new_question_jsons)

In [ ]:
for question_json in question_jsons:
  print(question_json)

[{'Questions': [{'Question': 'What is the main purpose of the FDA Amendments Act of 2007?', 'Hint': 'It gave the FDA authority to require certain information from manufacturers.', 'Answer': 'The FDA Amendments Act of 2007 gave the FDA authority to require a Risk Evaluation and Mitigation Strategy (REMS) from manufacturers to ensure that the benefits of a drug or biological product outweigh its risks.', 'Extra': None}, {'Question': 'What is the purpose of a boxed warning?', 'Hint': 'It is part of FDA-mandated labeling.', 'Answer': 'A boxed warning is a special type of warning that is required by the FDA to be placed on the label of a drug or biological product when there is a serious risk of adverse effects.', 'Extra': None}, {'Question': 'Which drugs are subject to special restrictions under federal and state law?', 'Hint': 'They include opioids and other drugs with a high potential for abuse.', 'Answer': 'Drugs that are subject to special restrictions include opioids, hallucinogens, s

In [ ]:
db.topic.create_index([("topic", "text")])

'topic_text'

In [ ]:
!pip install fuzzywuzzy

In [ ]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.9 MB/s eta 0:00:00


In [ ]:
from fuzzywuzzy import fuzz
def fuzzy_search_topic(topic_name, threshold=80):
    topics = db.topic.find()
    best_match = None
    highest_score = 0
    for topic in topics:
        score = fuzz.ratio(topic_name.lower(), topic['topic'].lower())
        if score > highest_score and score >= threshold:
            best_match = topic
            highest_score = score
    return best_match

In [ ]:
for question_json, topic in zip(question_jsons, topic_list):
    try:
       #topic_name = question_json[0]['Topic']
       #topic = db.topic.find_one({"topic": re.compile(f'^{topic_name}$', re.IGNORECASE)})

       #if not topic:
        #    topic = fuzzy_search_topic(topic_name)

       #print(topic)
       questions = question_json[0]['Questions']
       print(f"Topic_name: {topic['topic']}")
       print(f"Questions:{questions}")
       documents = convert_json_to_documents(topic, questions)
       docs.append(documents)
       #print(json.dumps(question_json[0], indent=4))
       #print("---------------------------------------------------------------------------------")
    except json.JSONDecodeError:
        print(f"Invalid JSON: {question}")
        print("---------------------------------------------------------------------------------")
    except (KeyError, TypeError) as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")
        print("---------------------------------------------------------------------------------")

Topic_name: Legal procedures
Questions:[{'Question': 'What is the main purpose of the FDA Amendments Act of 2007?', 'Hint': 'It gave the FDA authority to require certain information from manufacturers.', 'Answer': 'The FDA Amendments Act of 2007 gave the FDA authority to require a Risk Evaluation and Mitigation Strategy (REMS) from manufacturers to ensure that the benefits of a drug or biological product outweigh its risks.', 'Extra': None}, {'Question': 'What is the purpose of a boxed warning?', 'Hint': 'It is part of FDA-mandated labeling.', 'Answer': 'A boxed warning is a special type of warning that is required by the FDA to be placed on the label of a drug or biological product when there is a serious risk of adverse effects.', 'Extra': None}, {'Question': 'Which drugs are subject to special restrictions under federal and state law?', 'Hint': 'They include opioids and other drugs with a high potential for abuse.', 'Answer': 'Drugs that are subject to special restrictions include o

In [ ]:
for doc in docs:
  if(len(doc)> 0):
    results = db.viva_question_collection.insert_many(doc)
    print(f"Inserted {len(results.inserted_ids)} documents")

Inserted 10 documents
Inserted 10 documents
Inserted 10 documents
Inserted 10 documents


In [ ]:
print(len(docs))

4


In [ ]:
updated_results = db.viva_question_collection.update_many({'topic':'isaster & accident'},{'$set':{'topic':'Disaster & accident'}})
print(f"Updated {updated_results.modified_count} documents")

Updated 10 documents


In [ ]:
import re
topic_info = list(db.topic.find({"topic": re.compile(r"\s*Body fluids & semen analysis\s*")}))
topic_info

[{'_id': ObjectId('668cce2d449c846bbcf10ce8'),
  'prof': 'Third',
  'subject': 'Pathology',
  'card': 'Clinical Pathology',
  'topic': ' Body fluids & semen analysis',
  'subTopic': ' Body fluids & semen analysis'}]

In [ ]:
topic_name = topic_info[0]
topic_name

{'_id': ObjectId('668cce2d449c846bbcf10ce8'),
 'prof': 'Third',
 'subject': 'Pathology',
 'card': 'Clinical Pathology',
 'topic': ' Body fluids & semen analysis',
 'subTopic': ' Body fluids & semen analysis'}

In [ ]:
import json
# Function to convert JSON to a list of documents
def convert_json_to_documents(topic_name, question_json):
    documents = []
    for topic in question_json:
        for question in topic["Questions"]:
            doc = {
                'prof': topic_name['prof'],
                'subject': topic_name['subject'],
                'card': topic_name['card'],
                'topic': topic_name['topic'],
                'subTopic': topic_name['subTopic'],
                'question': question['Question'],
                'version': '1',
                'hint': question['Hint'],
                'correct_answer': question['Answer'],
                'extra': question['Extra'] if 'Extra'== None in question else "No exta info to add"
            }
            documents.append(doc)
    return documents

In [ ]:
db.viva_question_collection.insert_many(documents)

InsertManyResult([ObjectId('668d1728cd1952b10d20e447'), ObjectId('668d1728cd1952b10d20e448'), ObjectId('668d1728cd1952b10d20e449'), ObjectId('668d1728cd1952b10d20e44a'), ObjectId('668d1728cd1952b10d20e44b'), ObjectId('668d1728cd1952b10d20e44c'), ObjectId('668d1728cd1952b10d20e44d'), ObjectId('668d1728cd1952b10d20e44e'), ObjectId('668d1728cd1952b10d20e44f'), ObjectId('668d1728cd1952b10d20e450')], acknowledged=True)

In [ ]:
collection = db['viva_question_collection']

# Fields to extract
fields = ['subject', 'topic', 'question', 'hint', 'correct_answer']

# Query to get all documents
documents = collection.find({"subject":"Community Medicine"}, {field: 1 for field in fields})

# Convert documents to DataFrame
df = pd.DataFrame(list(documents), columns=fields)

In [ ]:
df

,subject,topic,question,hint,correct_answer
0,Community Medicine,Biostatistics,Describe the mechanism of action of opioids in...,Opioids interact with receptors in the central...,Opioids bind to opioid receptors in the centra...
1,Community Medicine,Biostatistics,What are the different types of opioid agonists?,There are three main types of opioid agonists.,The three main types of opioid agonists are fu...
2,Community Medicine,Biostatistics,How do opioids exert their respiratory depress...,Opioids affect the respiratory center in the b...,Opioids bind to opioid receptors in the respir...
3,Community Medicine,Biostatistics,What is the role of the NRTIs in the treatment...,NRTIs inhibit the reverse transcriptase enzyme.,NRTIs (nucleoside and nucleotide reverse trans...
4,Community Medicine,Biostatistics,What are the adverse effects associated with t...,"PPIs can cause several adverse effects, includ...",PPIs can cause adverse effects such as diarrhe...
...,...,...,...,...,...
113,Community Medicine,Disaster & accident,Why is vigabatrin use restricted in certain ca...,Potential adverse effect of vigabatrin,"Vigabatrin may cause irreversible visual loss,..."
114,Community Medicine,Disaster & accident,What is the purpose of calculating the anion g...,Significance of the anion gap,The anion gap helps identify metabolic acidosi...
115,Community Medicine,Disaster & accident,What electrolyte imbalance is commonly associa...,Effect of potassium-sparing diuretics on potas...,Potassium-sparing diuretics can cause hyperkal...
116,Community Medicine,Disaster & accident,What is the significance of detecting oxalate ...,Clinical implication of oxalate crystals in urine,The presence of large numbers of oxalate cryst...


In [ ]:
csv_file_path = 'community_medicine_questions.csv'

# Write DataFrame to CSV file
df.to_csv(csv_file_path, index=False)

print(f"CSV file '{csv_file_path}' has been created.")

CSV file 'community_medicine_questions.csv' has been created.


In [ ]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.4 MB/s eta 0:00:00


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

pdf_file_path = 'viva_questions.pdf'
doc = SimpleDocTemplate(pdf_file_path, pagesize=letter)
styles = getSampleStyleSheet()
story = []

# Add content to the PDF
for index, row in df.iterrows():
    story.append(Paragraph(f"Subject: {row['subject']}", styles['Heading2']))
    story.append(Paragraph(f"Topic: {row['topic']}", styles['Heading3']))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Question: {row['question']}", styles['BodyText']))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Hint: {row['hint']}", styles['BodyText']))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"Answer: {row['correct_answer']}", styles['BodyText']))
    story.append(Spacer(1, 24))

# Build the PDF
doc.build(story)

print(f"PDF file '{pdf_file_path}' has been created.")

PDF file 'viva_questions.pdf' has been created.


In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

pdf_filename = "questions.pdf"
c = canvas.Canvas(pdf_filename, pagesize=letter)
width, height = letter

# Define starting position for the text
x = 50
y = height - 50

# Set font and size
c.setFont("Helvetica", 12)

# Loop through the questions and add them to the PDF
for question in questions_list:
    c.drawString(x, y, question)
    y -= 20  # Move down by 20 units for the next question
    if y < 50:  # Check if the y position is too low for the next question
        c.showPage()  # Create a new page
        c.setFont("Helvetica", 12)  # Reset font for the new page
        y = height - 50  # Reset y position for the new page

# Save the PDF
c.save()

print(f"PDF generated: {pdf_filename}")

PDF generated: questions.pdf


#Run on OpenAI

In [32]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.3 MB/s eta 0:00:00


In [40]:
query = "Head"
knowledge = get_knowledge(query, book_collection)
context = get_search_result(knowledge)
prompt = get_prompt_viva(query=query, context=context)
prompt

[0.003369350917637348, -0.009024985134601593, -0.005425000563263893, -0.027685411274433136, -0.002223437884822488, -0.003325390163809061, 0.00783743616193533, 0.00933422613888979, 0.039363179355859756, 0.00585462711751461, 0.029920395463705063, -0.03179478272795677, 0.027497753500938416, -0.01880066841840744, -0.012361550703644753, 0.02306537516415119, -0.037682950496673584, -0.025275902822613716, -0.010663425549864769, -0.004175587557256222, 0.020106693729758263, 0.00731809064745903, -0.06821944564580917, -0.01675819419324398, -0.020977161824703217, 0.02612103894352913, 0.025822820141911507, -0.010245281271636486, 0.07745673507452011, 0.059794917702674866, -0.03603480011224747, -0.021086491644382477, 0.030742177739739418, -0.0776183232665062, 0.004147338680922985, -0.033967189490795135, 0.03215474262833595, -0.054670535027980804, -0.027509571984410286, -0.044833336025476456, 0.028977414593100548, -0.002864697016775608, -0.006569428835064173, -0.03604865446686745, -0.016304001212120056

'You are assisting a medical student in preparing for an exam. Your task is to generate a viva question based on the given MCQ, Multiple Choice Question, Single Best answer, SQA question and True-False Question using medical terminology. Ensure your responses are comprehensive and friendly, reflecting a conversational tone. Your only focus is to generate Viva Question and you are given the Question Format. Make sure to generate questions in full sentence. Your task is to create the viva question.  Question should be answerable in 1-2 sentences. You have 5 tasks here: 1. You can generate as many questions you can. 2. You should provide corresponding hints for those questions. 3. You should also provide answers for those questions. Answers should be around 2-3 sentences. 4. You must provide extra information for the questions. If you do not find anything to add give null 5. Output should be a perfect json object with proper format. Do not add any extra characters. The output should be a 

In [42]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OpenAIKey')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [59]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Tell me This is something",
        }
    ],
    model="gpt-3.5-turbo",
)

In [60]:
chat_completion

ChatCompletion(id='chatcmpl-9kmFBvetEpeNJ1QEMmUwrDxh4OjRT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='This is something.', role='assistant', function_call=None, tool_calls=None))], created=1720935565, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=4, prompt_tokens=12, total_tokens=16))

In [55]:
import json

# Function to convert JSON to a list of documents
def convert_json_to_documents(topic_name, questions):
    documents = []
    for question in question_json:
        if len(questions) > 0:
            for question in questions:
                doc = {
                    'prof': topic_name['prof'],
                    'subject': topic_name['subject'],
                    'card': topic_name['card'],
                    'topic': topic_name['topic'],
                    'subTopic': topic_name['subTopic'],
                    'question': question['Question'],
                    'version': '2',
                    'hint': question['Hint'],
                    'correct_answer': question['Answer'],
                    'extra': question.get('Extra', "No extra info to add")
                }
                documents.append(doc)
    return documents

In [56]:
import re
import json

def extract_json(text):
    # Regex to match JSON objects
    json_regex = re.compile(r'\{(?:[^{}]*|\{(?:[^{}]*|\{[^{}]*\})*\})*\}')

    # Function to recursively parse JSON objects
    def parse_json(match):
        try:
            json_object = json.loads(match)
            # Check if there are nested JSON objects
            for key, value in json_object.items():
                if isinstance(value, str) and re.match(json_regex, value):
                    json_object[key] = parse_json(value)
            return json_object
        except json.JSONDecodeError:
            return None

    # Find all JSON objects in the text
    json_matches = json_regex.findall(text)

    # Parse the JSON objects
    json_objects = []
    for json_match in json_matches:
        json_object = parse_json(json_match)
        if json_object is not None:
            json_objects.append(json_object)

    return json_objects

In [ ]:
import time
import json

questions_list = []
input_token_count = 0
output_token_count = 0
for topic in topic_list[:2]:
    query = topic['topic']

    try:
        flag = False
        #for topic in question_topics:
         #   if topic == query:
          #      flag = True
        # Get the context and prompt
        if flag == False:
          knowledge = get_knowledge(query, book_collection)
          context = get_search_result(knowledge)
          chat_completion = client.chat.completions.create(
          messages=[
               {
                "role": "user",
                "content": prompt,
              }
           ],
            model="gpt-3.5-turbo",
           )
          input_token_count += chat_completion.usage.prompt_tokens
          output_token_count += chat_completion.usage.completion_tokens
          question = chat_completion.choices[0].message.content
          questions_list.append(question)
    except ValueError as e:
        print(f"Error generating content for topic '{topic}': {e}")
        continue

    # Add a 10-second wait time
    time.sleep(20)

In [63]:
total_cost = input_token_count*(0.0015/1000) + output_token_count*(0.002/1000)
print(f"Total cost: {total_cost}")

Total cost: 0.003879


In [64]:
query = {"flag": 1}

    # Update the flag to 0
new_values = {"$set": {"flag": 0}}

    # Perform the update
results = db.Medical_Books_RAG.update_many(query, new_values)
print(f"Updated {results.modified_count} documents")

Updated 0 documents


In [65]:
docs = []
question_jsons = []
for question in questions_list:
        question_text = f"[{question}]"
        new_question_jsons = extract_json(question_text)
        question_jsons.append(new_question_jsons)

In [66]:
for question_json, topic in zip(question_jsons, topic_list):
    try:
       questions = question_json[0]['Questions']
       print(f"Topic_name: {topic['topic']}")
       print(f"Questions:{questions}")
       documents = convert_json_to_documents(topic, questions)
       docs.append(documents)
    except json.JSONDecodeError:
        print(f"Invalid JSON: {question}")
    except (KeyError, TypeError) as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")

Topic_name: Legal procedures
Questions:[{'Question': 'What is the main function of the cerebrum?', 'Hint': 'Think about the largest part of the brain responsible for voluntary actions, thoughts, and memory.', 'Answer': 'The cerebrum controls voluntary movements, thoughts, and memories.', 'Extra': None}, {'Question': 'Which cranial nerve is responsible for the sense of smell?', 'Hint': 'Consider the cranial nerves associated with sensory functions.', 'Answer': 'Cranial nerve I, the olfactory nerve, is responsible for the sense of smell.', 'Extra': None}, {'Question': 'What is the function of the occipital lobe?', 'Hint': 'Think about the specific functions associated with the occipital lobe at the back of the brain.', 'Answer': 'The occipital lobe is primarily responsible for visual processing.', 'Extra': None}, {'Question': 'Which structures protect the brain from mechanical injury?', 'Hint': 'Consider the bony structures that surround and protect the brain.', 'Answer': 'The skull and 

In [69]:
for doc in docs:
  print(doc)

[{'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Legal procedures', 'subTopic': 'Legal procedures', 'question': 'What is the main function of the cerebrum?', 'version': '2', 'hint': 'Think about the largest part of the brain responsible for voluntary actions, thoughts, and memory.', 'correct_answer': 'The cerebrum controls voluntary movements, thoughts, and memories.', 'extra': None}, {'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Legal procedures', 'subTopic': 'Legal procedures', 'question': 'Which cranial nerve is responsible for the sense of smell?', 'version': '2', 'hint': 'Consider the cranial nerves associated with sensory functions.', 'correct_answer': 'Cranial nerve I, the olfactory nerve, is responsible for the sense of smell.', 'extra': None}, {'prof': 'Second', 'subject': 'Forensic Medicine', 'card': 'Term-1 ', 'topic': 'Legal procedures', 'subTopic': 'Legal procedures', 'question': 'What is the function of the

In [80]:
from tqdm import tqdm

for doc in tqdm(docs, desc="Inserting Documents"):
  if(len(doc)> 0):
    results = db.viva_question_collection.insert_many(doc)

print(f"Inserted {len(results.inserted_ids)} documents")

Inserting Documents: 100%|██████████| 2/2 [00:00<00:00, 29.67it/s]

Inserted 10 documents
Inserted 10 documents
